# Lab session 1: 1D Continous Wavelet Transform

## Prerequisite

### Install Anaconda and PyWavelet

* The simplest is to download the Anaconda distribution (including PyWavelet) : https://www.anaconda.com/ 
* Or follows these instructions: https://pywavelets.readthedocs.io/en/latest/install.html
* Read the documentation on Pywavelet: https://pywavelets.readthedocs.io/_/downloads/en/v0.5.1/pdf/
* Code available on Github (to know how it is implemented): https://github.com/PyWavelets/pywt/tree/580d79d9440ec0f4f936892e39c79ad13a8fd33b 

### Instructions

* Fill empty codes and answer the questions in the notebook
* Drop on TEIDE an archive (with your NAME in capital letters, separated by an underscore for teams NAME1_NAME2) containing:
    * your completed notebook (the python code .ipynb)
    * the corresponding exported PDF
* **Deadline: November 13 (23h59)**. 

### Mark scheme

* This practical work is graded out of 20 points (but at the end the two lab sessions will represent a quarter of the final mark, i.e 2,5 points each)
* Hand in overdue: -10% per day
* Plagiarism: mark divided by 2 (copy/paste from the internet or other groups is forbidden)

### Packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import pywt
from pylab import*
import scaleogram as scg

## Fourier transform limitation in a nutschell

**Question 1** *(2 points)*: Let's compute the Fourier transform of 2 signals made of
1. The superposition of 4 sinusoids with different frequencies on $[0,1]$ (notes played together)
2. The concatenation of these 4 sinusoids each with duration $1/4$ on $[0,1]$ (notes played successively)

**Answer 1**:

In [ ]:
def fft_compute(y, T, N):
    f = np.linspace(0.0, 1.0/(2.0*T), N//2)
    fft_y = fft(y)
    fft_y = 2.0/N * np.abs(fft_y[0:N//2])
    return f, fft_y

N = 1000
T = 1.0/N
 
x = np.linspace(0, 1, num=N)
 
freq = [8, 20, 35, 50]
np.random.shuffle(freq)

# ANSWER :
y_superp = np.sum([np.sin(2 * np.pi * freq[i] * x) for i in range(4)], axis=0)
sub_array = np.split(x, 4)
y_concat = np.concatenate([np.sin(2 * np.pi * freq[i] * sub_array[i]) for i in range(4)])
 
f1, fft_superp = fft_compute(y_superp, T, N)
f2, fft_concat = fft_compute(y_concat, T, N)

plt.figure(figsize=(12, 6))
plt.subplot(221)
plt.plot(x, y_superp)
plt.title('signal from superposition')
plt.xlabel('time')

plt.subplot(223)
plt.plot(x, y_concat)
plt.title('signal from concatenation')
plt.xlabel('time')

plt.subplot(222)
plt.plot(f1, fft_superp, 'r')
plt.xlim(0, 60)
plt.title('FFT of the superposition signal')
plt.xlabel('frequency')

plt.subplot(224)
plt.plot(f2, fft_concat, 'r')
plt.xlim(0, 60)
plt.title('FFT of the concatenation signal')
plt.xlabel('frequency')
 
plt.tight_layout()
plt.show()

**Question 2** *(2 points)*: 
* Explain why the peaks on the concatenate version is more spread
* Try to exchange the order of the concatenation. Conclusion?

**Answer** 
First, we observe that the concatenated signal is not regular (the superposed signal is regular). It could also be interpreted as the sum of windowed signal leading to sinus cardinal (sinc) spread on the spectrogram. 

After exchange order of concatenation :

For the concatenation spectrogram, we observe that is is not always the very same results, but it looks similar (the dominants spikes remains at the same place). This can be explained by the presence of irregularities between each window (we tried with only one frequency on 1/4 of the signal, in this case only the sinc deformation appear)

## Some signals to play with

The *PyWavelet package* provides a list of signals demo:

In [ ]:
signals = pywt.data.demo_signal('list')

print(signals)
print(len(signals))

To get the values of a signal and display it:

In [ ]:
signal_length = 1024
y = pywt.data.demo_signal('Piece-Regular', signal_length)
plt.figure(figsize=(12, 6))
plt.plot(y)
plt.show()

**Question 3** *(2 points)*: Display in a loop the others signal in different subplots. You will especially make appear both the real and imaginary parts of the Gabor signal.

In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=5)
fig.set_size_inches(25, 15)

# plot 15 first signals :
for i in range(3):
    for j in range(5):
        axs[i, j].plot(pywt.data.demo_signal(signals[i * 5 + j], signal_length))
        axs[i, j].set_title(signals[i * 5 + j])

axs[3, 0].plot(pywt.data.demo_signal('Piece-Regular', signal_length))
axs[3, 0].set_title('Piece-Regular')

axs[3, 1].plot(pywt.data.demo_signal('Piece-Polynomial', signal_length))
axs[3, 1].set_title('Piece-Polynomial')

axs[3, 2].plot(pywt.data.demo_signal('Riemann', signal_length))
axs[3, 2].set_title('Riemann')

gabor = pywt.data.demo_signal('Gabor')
axs[3, 3].plot(np.real(gabor))
axs[3, 3].set_title('Gabor real')
axs[3, 4].plot(np.imag(gabor))
axs[3, 4].set_title('Gabor imaginary')


plt.show()



## Familiarize yourself with the family of continuous wavelets

Display the family of all wavelets available in *PyWavelet*:

In [ ]:
print(pywt.families(short=False))

For such a family (for instance the *Debauchies wavelets*), all the available wavelets are given by:

In [ ]:
print(pywt.wavelist('db')) # Daubechies wavelets

The **continuous wavelet** are given by:

In [ ]:
print(pywt.wavelist(kind='continuous'))

To read the characteristics of a particular wavelet:

In [ ]:
w = pywt.ContinuousWavelet('gaus2')
print(w)

### Vizualize some classical continuous wavelets

**Question 4** *(1 points)*: Display in a loop all the continous wavelets in different subplots.

*Hint: To compute an approximation of the scaling function (psi) use the command `wavefun` (see documentation for more details: https://pywavelets.readthedocs.io/en/latest/ref/wavelets.html)*

In [ ]:
continuous_wavelets = pywt.wavelist(kind='continuous')
level = 10
print(len(continuous_wavelets))

fig, axs = plt.subplots(nrows=5, ncols=5)
fig.set_size_inches(25, 15)

# plot 15 first signals :
for i in range(5):
    for j in range(5):
        if i * 5 + j < 21:
            w = pywt.ContinuousWavelet(continuous_wavelets[i * 5 + j])
            psi, x = w.wavefun(level=level)
            axs[i, j].plot(x, psi)
            axs[i, j].set_title(continuous_wavelets[i * 5 + j])

plt.show()


### Vizualize the scaling of the Morlet wavelet in time and frequency domains

**Question 5** *(3 points)*: Plot wavelets at different scales from a given mother wavelet, as well as their Fourier transforms

To vizualize the mother wavelet one can use the `scaleogram` package (see the documentation https://github.com/alsauve/scaleogram/blob/master/doc/tests.ipynb and https://github.com/alsauve/scaleogram/blob/master/doc/scale-to-frequency.ipynb):

In [ ]:
time_scale = ['0.1', '0.5', '1', '5', '10']
freq_scale = ['0.2', '0.5', '1', '2', '5']

for i in range(5):
    axes = scg.plot_wav('cmor' + time_scale[i] + '-' + freq_scale[i], figsize=(14,3))



## The 1D Continuous Wavelet Transform (CWT)

### Compute the CWT

The function `pywt.cwt` enables to compute the CWT of a discrete signals at different scales.

See the documentation: https://pywavelets.readthedocs.io/en/latest/ref/cwt.html

In theory the CWT is defined in a continuous domain. But in practise, we always deal with input discretized signals and compute the wavelet transform for specific given scales which are also sampled. Then numerically we compute an approximation of the CWT of a discrete signal $\mathbf{f}=\{f_i\}_{i\in \mathbb{Z}}$, obtained by sampling a continuous function $f^0$ on a regular discrete grid $\{x_i\}_{i\in \mathbb{Z}}$ that is $f_i=f^0(x_i)$ and $x_{i+1}=x_{i}+\Delta$. Then we consider the piecewise constant function $f$ defined by 

$$ f(x)=\sum_i f_i \chi_{[x_i, x_{i+1}]} $$

Therefore, the CWT of the continuous function $f$ is given by:

\begin{align}
    Wf(a,b) & = \frac{1}{\sqrt{a}}\int_{\mathbb{R}} f(x) \overline{\psi\left(\frac{x-b}{a}\right)}\mathrm{d}x \\
            & = \frac{1}{\sqrt{a}}\sum_{i\in \mathbb{Z}} f_i \int_{x_i}^{x_{i+1}} \overline{\psi\left(\frac{x-b}{a}\right)}\mathrm{d}x \\
            & = \frac{1}{\sqrt{a}}\sum_{i\in \mathbb{Z}} f_i \left(\int_{-\infty}^{x_{i+1}} \overline{\psi\left(\frac{x-b}{a}\right)}\mathrm{d}x - \int_{-\infty}^{x_{i}} \overline{\psi\left(\frac{x-b}{a}\right)}\mathrm{d}x\right) \\
            & = \sqrt{a}\sum_{i\in \mathbb{Z}} f(x_i) \left(\overline{\psi_{\mathrm{int}}^a(x_{i+1}-b)}-\overline{\psi_{\mathrm{int}}^a(x_{i}-b)}\right) \\
            & = \sqrt{a}\sum_{i\in \mathbb{Z}} f(x_i) \left(\overline{\psi_{\mathrm{int}}^a(x_{i}-(b-\Delta))}-\overline{\psi_{\mathrm{int}}^a(x_{i}-b)}\right) \\
            & = -\sqrt{a}\left[ (f\ast \psi_{\mathrm{int}}^a)(b) - (f\ast \psi_{\mathrm{int}}^a)(b-\Delta)\right]
\end{align}

with

$$ \psi_{\mathrm{int}}^a(y) \overset{\mathrm{def}}{=} \frac{1}{a} \int_{-\infty}^y \psi\left(\frac{x}{a}\right) \mathrm{d}x = \int_{-\infty}^{y/a} \psi\left(u\right) \mathrm{d}u = \psi_{\mathrm{int}}^1\left(\frac{y}{a}\right) $$

If the translation parameter $b$ is sampled on the same grid $\{x_i\}_{i\in \mathbb{Z}}$ then we get:

$$ Wf(a, \{x_i\}) = -\sqrt{a}\, \mathrm{diff}\, [ \mathbf{f} \star \psi_{\mathrm{int}}^a(\{x_i\}) ] $$

**Question 6** *(3,5 points)*: Apply the CWT to reproduce the 6 examples given in the course:
* Example 1: pure cosine
* Example 2: a Dirac
* Example 3: the periodic square wave
* Example 4: a modulated wave
* Example 5: 2 sinusoids with noise
* Example 6: Holder function of exponant 1/2

*Hint:* Consider for the first example a cosine with frequency $f=50~Hz$ on $[0,1]$ with a sampling rate $f_s=\frac{1}{400}$, compute its CWT onto scales from 1 to 50 and display the corresponding scalogram (by plotting the coef/freqs in output of `pywt.cwt`). Check that you vizually detect the frequency $f$.

In [ ]:
from scipy import signal

N = 400 # sampling rate in time
x = np.linspace(0, 1, N)
freq = 50 # main frequency for following signals

functions = [
    (np.cos(2 * np.pi * freq * x), 'cos'),
    (np.concatenate([np.ones((1)), np.zeros(len(x) - 1)]), 'Dirac'),
    (signal.square(2 * np.pi * freq / 6 * x), 'Square Wave'),
    (signal.chirp(x, f0=50, f1=0.5, t1=1, method='linear'), 'Frequenced modulated wave'),
    (np.sin(2 * np.pi * freq * x) + np.sin(2 * np.pi * (freq / 2) * x) + np.random.normal(0, 0.3, size=N), 
    '2 sin with noise'),
    (np.sqrt(np.abs(np.cos(2 * np.pi * 2 * x))), 'Holder function of exponant 1/2') 
    # doubled the frequency of Holder fct to have a better overview
]

# print signals :
n = len(functions)
fig, axs = plt.subplots(nrows=len(functions), ncols=1)
fig.set_size_inches(20, 20)
for i in range(n):
    axs[i].plot(x, functions[i][0])
    axs[i].set_title(functions[i][1])

# plot scalogram :
scale = np.arange(1, 50)
fig, axs = plt.subplots(nrows=len(functions), ncols=1)
fig.set_size_inches(20, 20)
fig.tight_layout(pad=2.2)
for i in range(n):
    coefs, frequencies = pywt.cwt(
        functions[i][0], scale, 'morl', sampling_period=1/N)
    axs[i].matshow(coefs, extent=[0, 1, 1, 50], cmap='PRGn', aspect='auto', vmax=abs(
        coefs).max(), vmin=-abs(coefs).max())
    axs[i].set_title("Scalogram of " + functions[i][1])


**Question 7** *(2 points)*: Same question applied to both examples of Question 1 (superposition and concatenation). Can you detect all the frequencies and their localization?

In [ ]:
# y_concat, y_superp
freq = [10, 25, 50, 100] # change the frequency in order to see well each freq
y_superp = np.sum([np.sin(2 * np.pi * freq[i] * x) for i in range(4)], axis=0)
sub_array = np.split(x, 4)
y_concat = np.concatenate(
    [np.sin(2 * np.pi * freq[i] * sub_array[i]) for i in range(4)])

functions = [(y_concat, 'concat sin'),
            (y_superp, 'superp sin')]
fig, axs = plt.subplots(nrows=2, ncols=1)
fig.set_size_inches(10, 20)
fig.tight_layout(pad=2.2)
for i in range(2):
    coefs, frequencies = pywt.cwt(
        functions[i][0], scale, 'morl', sampling_period=1/N)
    axs[i].matshow(coefs, extent=[0, 1, 1, 50], cmap='PRGn', aspect='auto', vmax=abs(
        coefs).max(), vmin=-abs(coefs).max())
    axs[i].set_title("Scalogram of " + functions[i][1])


**Question 8** *(3 points)*: After applying the CWT to the previous demo-signal 'Piece-Regular', you will emulate the maxima lines by finding local peaks at each scale (without chain them)

Suggestions:
* https://github.com/MonsieurV/py-findpeaks
* https://github.com/buckie/wtmm-python

In [ ]:
signal_length = 1024
piece_regular = pywt.data.demo_signal('Piece-Regular', signal_length)
scale = np.arange(1, 50)

coefs, frequencies = pywt.cwt(
    piece_regular, scale, 'morl', sampling_period=1/N)

# Peaks search 
maxima_lines = []
for c in coefs:
    indexes, _ = signal.find_peaks(c)
    maxed = np.zeros(shape=(len(c),))
    for i in indexes:
        maxed[i] = c[i]
    maxima_lines.append(maxed)
maxima_lines = np.array(maxima_lines)

# Peaks scaleogram
fig, ax = plt.subplots()
fig.set_size_inches(15, 8)
fig.tight_layout(pad=2.2)
ax.matshow(maxima_lines, extent=(0, 1024, 1, 200), cmap='PRGn', aspect='auto', vmax=abs(
    maxima_lines).max(), vmin=-abs(maxima_lines).max())
ax.set_title("Scalogram of Piece-Regular")

**Question 9** *(1,5 points)*: Compute the CWT of a fractional Brownian motion with Hölder exponent 1/2

Use the `fbm` package: https://github.com/crflynn/fbm 



In [ ]:
from fbm import FBM

f = FBM(n=1024, hurst=0.75, length=1, method='daviesharte')
# Generate a fBm realization
fbm_sample = f.fbm()

plot(fbm_sample, label="fbm")
exponant = 1/2
fbm_holder = np.power(np.abs(fbm_sample), exponant)
plot(fbm_holder, label="hölder fbm (exponant " + str(exponant) + ")")

legend()
grid()
title('Fractional Brownian')
show()

# fbm
coefs_fbm, frequencies_fbm = pywt.cwt(
    fbm_sample, scale, 'morl', sampling_period=1/1024)
matshow(coefs_fbm, extent=[0, 1, 1, 200], cmap='PRGn', aspect='auto', vmax=abs(
        coefs_fbm).max(), vmin=-abs(coefs_fbm).max())

# holder fbm
coefs_fbm_holder, frequencies_fbm_holder = pywt.cwt(
    fbm_holder, scale, 'morl', sampling_period=1/1024)
matshow(coefs_fbm_holder, extent=[0, 1, 1, 200], cmap='PRGn', aspect='auto', vmax=abs(
        coefs_fbm_holder).max(), vmin=-abs(coefs_fbm_holder).max())


**Question 10 (Bonus)** *(2 points)*: Recover the Hölder exponent by computing the slope of decrease of the wavelet coefficients.

In [ ]:
a = np.flip(np.abs(coefs_fbm).mean(axis=1)) # apply |.|1 norm to coefficients
b = np.flip(np.abs(coefs_fbm_holder).mean(axis=1))

plot(a, label="fbm")
plot(b, label="fbm_holder")
legend()
ylabel("Absolute mean of wavelets coefficients")
xlabel("Scale (frequency)")

# linear regression to find the slope
slope_fbm, b_fbm = np.polyfit(np.arange(
    a.shape[0]), a, 1)  # 1 = degree of polynomial
slope_fbm_holder, b_fbm_holder = np.polyfit(np.arange(b.shape[0]), b, 1)

print("Fbm slope :", slope_fbm)
print("Holder fbm slope :", slope_fbm_holder)

# Did not manage to recover the 1/2 coefficient
